In [1]:
import os
import io
import re
import oeis
import functools
import torch


In [2]:


path = '/Users/nolte/Dropbox/ML'


def readESymb(loop, path, file):
    name = os.path.join(path, file)
    assert os.path.isfile(name), name
    res = ''
    prefix = 'Esymb'
    with open(name, 'rt') as f:
        reading_form = False
        for line in f:
            if not reading_form:
                if not line.startswith(prefix + '[' + str(loop) + ']'): continue
                res = ''
                reading_form = True
            res += line[:-2] if line[-2] == '\\' else line[:-1]
            if line[-2] in [":", ";"]:
                break
    return res


def convert(loop):
    file = 'EZ6_symb_new_norm' if loop == 6 else 'EZ_symb_new_norm'
    s = re.split(":=|SB\(|\)", re.sub('[,*]', '', readESymb(loop, path, file)))[1:-1]
    keys = s[1::2]
    values = [int(re.sub('[+-]$',t[0]+'1',t)) for t in s[0::2]]
    return {k:v for k,v in zip(keys,values)}



def query_key(key,verbose=False):
    l = len(key)
    if l==0:
        if verbose: print('empty key')
        return 0
    if l%2==1:
        if verbose: print("odd length key")
        return 0
    if key[0] in ['d', 'e', 'f'] or key[-1] in ['a', 'b', 'c']:
        if verbose: print("stupid zero: begin/end")
        return 0
    for i in range(l-1):
        if (key[i] == 'a' and key[i+1] == 'd') \
            or (key[i] == 'b' and key[i+1] == 'e') \
            or (key[i] == 'c' and key[i+1] == 'f'):
                if verbose: print('stupid zero: adjacency')
                return 0
        if (key[i] == 'd' and key[i+1] in ['a','e','f'])\
            or (key[i] == 'e' and key[i+1] in ['b','d','f'])\
            or (key[i] == 'f' and key[i+1] in ['c','d','e']):
                if verbose: print('stupid zero: adjacency')
                return 0


    loop  = l//2
    if loop < 1 or loop > 6:
        print("loop:", loop, "1 to 6 only:")
        return 0
    if key in data[loop].keys():
        return data[loop][key]
    return 0



data = [[]]
print("loading...")
for i in range(1,7):
    e=convert(i)
    data.append(e)
print("loaded")

for i in range(7):
    print(i, len(data[i]))



loading...
loaded
0 0
1 6
2 12
3 636
4 11208
5 263880
6 4916466


In [3]:
import numpy as np

def gen_abcseq(size):
    return ''.join(np.random.choice(['a','b','c'],size))

def add_run(seq,size):
    d = seq[-1]
    if d== 'a':
        l = ['e','f']
    elif d== 'b':
        l = ['d','f']
    else:
        l = ['d','e']
    return seq + str(np.random.choice(l))*size

def add_abcseq(seq,size):
    d = seq[-1]
    if d== 'd':
        l = ['b','c']
    elif d== 'e':
        l = ['a','c']
    else:
        l = ['a','b']
    return seq + np.random.choice(l) + gen_abcseq(size-1)

def build_from_struct(struct):
    s = gen_abcseq(struct[0])
    i = 1
    while i < len(struct):
        s=add_run(s,struct[i])
        i+=1
        if i == len(struct):
            break
        s = add_abcseq(s,struct[i])
        i += 1
    return s

def rand_key(loop):
    l = 2 * loop
    s = []
    while l > 0:
        n = np.random.choice(range(1,l))
        s.append(n)
        l -= n
        if l == 1:
            s.append(1)
            break
        n = np.random.choice(range(1,l))
        if n == l-1:
            s.append(l)
            break
        s.append(n)
        l -= n
    # print(s)
    return build_from_struct(s)

def change_last_run(seq):
    for n in range(-1,-len(seq), -1):
        if seq[n] in ['a','b','c']:
            break
    beg = seq[:n+1]
    if seq[n] == 'a':
        ch = 'e' if seq[n+1] == 'f' else 'f'
    elif seq[n] == 'b':
        ch = 'd' if seq[n+1] == 'f' else 'f'
    else:
        ch = 'e' if seq[n+1] == 'd' else 'd'
    return beg + ch*(-1-n), -1-n

def extract_signature(key):
    sig = []
    pos = 0
    while pos < len(key):
        posb=pos
        while key[pos] in ['a','b','c']:
            pos += 1
        sig.append(key[posb:pos])
        posb=pos
        pos+=1
        while pos < len(key) and key[pos] == key[posb]:
            pos += 1
        sig.append(key[posb:pos])
    return sig

def extract_subsequences(key):
    sig = extract_signature(key)
    res = []
    for i in range(len(sig)//2):
        res.append(sig[2*i]+sig[2*i+1])
    return res

stats = []
for i in range(12):
    stats.append([0,0,0,0])

for i in range(1000000):
    e = rand_key(6)
    f,n = change_last_run(e)
    ec = query_key(e)
    fc = query_key(f)
    if ec == fc and fc == 0:
        stats[n][0] += 1
    elif ec == fc:
        stats[n][1] += 1
    elif ec == -fc:
        stats[n][2] += 1
    else:
        stats[n][3] += 1

    #print(e, extract_signature(e), query_key(e))
    #r = extract_subsequences(e)
    #for t in r:
for i in range(12):
    print(i,stats[i])

struct = [3,1,2,4]

seqs=[]

for i in range(1):
    seqs.append(build_from_struct(struct))

#print(seqs)

for i in range(1):
    e1 = query_key(seqs[i])
    e2 = query_key(change_last_run(seqs[i]))
    if e1 != -e2:
        print (e1, e2, seqs[i],change_last_run(seqs[i]))
#    print(seqs[i],change_last_run(seqs[i]))
#    print(query_key(seqs[i]),query_key(change_last_run(seqs[i])))
print('done')


0 [0, 0, 0, 0]
1 [238097, 261809, 0, 0]
2 [16688, 0, 237689, 0]
3 [2372, 4694, 836, 79870]
4 [1053, 766, 451, 43713]
5 [544, 599, 231, 28279]
6 [281, 600, 406, 20041]
7 [144, 627, 100, 15696]
8 [55, 1518, 1133, 10834]
9 [0, 3902, 0, 7581]
10 [10270, 0, 0, 9121]
11 [0, 0, 0, 0]
-96 0 acafabffff ('acafabdddd', 4)
done


In [4]:
def prime_factors(n):
    if n == 0:
        return {}
    factors = []
    spf = [0 for i in range(n+1)]
    spf[1] = 1
    for i in range(2, n+1):
        spf[i] = i
    for i in range(4, n+1, 2):
        spf[i] = 2

    for i in range(3, int(n**0.5)+1):
        if spf[i] == i:
            for j in range(i*i, n+1, i):
                if spf[j] == j:
                    spf[j] = i

    while n != 1:
        factors += [spf[n]]
        n = n // spf[n]
    return dict(zip(*np.unique(factors, return_counts=True)))


In [5]:
word = "ae"
sequence = []
for i in range(5):
  word = "a" + word + "e"
  sequence.append(query_key(word, verbose=True))


In [6]:
print(sequence)
print([prime_factors(abs(s)) for s in sequence])


[0, -56, -64, -4128, -372224]
[{}, {2: 3, 7: 1}, {2: 6}, {2: 5, 3: 1, 43: 1}, {2: 9, 727: 1}]


In [7]:
sequence


[0, -56, -64, -4128, -372224]

In [8]:
#oeis.query(sequence)
oeis.query([0, -56, -64])[0].sequence


[0, 0, 0, 0, 0, 0, 56, 64, 448, 1552, 8952, 65120, 284584, 1491800, 8467816]

In [9]:
[x for x,y in data[4].items() if y == 448]


['bbbffbbf',
 'bbbddbbf',
 'cccddcce',
 'bbbddbbd',
 'aaaeeaae',
 'aaafaaff',
 'ccceeccd',
 'cccddccd',
 'cccdccdd',
 'aaaeaaee',
 'bbbfbbff',
 'ccceecce',
 'bbbffbbd',
 'ccceccee',
 'aaaffaaf',
 'aaaffaae',
 'bbbdbbdd',
 'aaaeeaaf']

In [10]:
def find_sequence(seq):
  elements = dict()
  for i,s in enumerate(seq):
    elements[i+1] = [x for x,y in data[i+1].items() if abs(y) == abs(s)]
  return elements


In [11]:
@functools.cache
def cached_oeis_query(query):
  return oeis.query(query)


In [12]:
query_key("bbbbbfffff")


-4128

In [226]:
find_sequence([0, 0, 56, 64, 448, 1552])


{1: [],
 2: [],
 3: ['bbbfff', 'aaafff', 'ccceee', 'aaaeee', 'bbbddd', 'cccddd'],
 4: ['bbaafbbf',
  'aaaffbbf',
  'bbaffbbf',
  'aeafffbf',
  'bbccdbbf',
  'bdbfbbbf',
  'cdbfbbbf',
  'aecdbbbf',
  'babffbbf',
  'abbffbbf',
  'abaffbbf',
  'baaffbbf',
  'aabafbbf',
  'aaabfbbf',
  'aeafbbbf',
  'ceafbbbf',
  'cccddbbf',
  'bbcddbbf',
  'aaaafbbf',
  'ccbcdbbf',
  'bbcbdbbf',
  'bbbcdbbf',
  'bbccdccd',
  'bbcbdccd',
  'aacceccd',
  'ccaeeccd',
  'bbaaffbd',
  'abfbffbd',
  'baffbfbd',
  'bbabffbd',
  'abffbfbd',
  'bafbffbd',
  'bccddcce',
  'bbbddcce',
  'cbcddcce',
  'bcbddcce',
  'bbcddcce',
  'cccbdcce',
  'cdcddcce',
  'ccbddcce',
  'bfbdccce',
  'afbdccce',
  'bddddcce',
  'cddddcce',
  'aaacbfff',
  'bfbaffff',
  'afbaffff',
  'bbbbffff',
  'ccacbfff',
  'aacabfff',
  'ccacafff',
  'afaecddd',
  'ccaaecdd',
  'ccafbbbd',
  'ccbfbbbd',
  'ccbbdbbd',
  'aeafbbbd',
  'ceafbbbd',
  'cccddbbd',
  'bbcddbbd',
  'aaaafbbd',
  'ccbcdbbd',
  'bbcbdbbd',
  'bbbcdbbd',
  'aaafbbdd',
  'ab

In [280]:
def grow(seq):
  keys, vals = seq
  order = len(keys[-1]) // 2
  for key, val in data[order + 1].items():
    # if all(keys[-1].count(x) <= key.count(x) for x in ['a','b','c','d','e','f']):
    vals_cand = vals + (val,)
    if len(cached_oeis_query(vals_cand)) > 0:
      yield (keys + (key,), vals_cand)


In [286]:
#seq = (("ae",), (-2,))
#seq = (('ae', 'aaae', 'aaaaae'), (-2, 16, -384))
seq = (('ae', 'aaae', 'aaaeae', 'aeaeaea'), (-2, 16, 28, 160)) # A321308
#seq = (('ae', 'aaae', 'aaaeae', 'bcbabfbd'), (-2, 16, 28, -8)) # A320148

#list(grow(seq))
list(grow(seq))


KeyboardInterrupt: 

In [314]:
# make an RNN
class ResBlock(torch.nn.Module):
    def __init__(
        self,
        d_model,
        hidden_dim=None,
    ) -> None:
        super().__init__()
        hidden_dim = hidden_dim or d_model
        self.fc1 = torch.nn.Linear(d_model, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, d_model)
        self.act = torch.nn.ReLU()

    def forward(self, x):
        x = self.fc2(self.act(self.fc1(x))) + x
        return x


class RNN(torch.nn.Module):
  def __init__(self, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = torch.nn.Embedding(6, d_model)
    self.layer = ResBlock(d_model)

  def forward(self, x):
    _, L = x.shape
    x = self.embedding(x)
    for _ in range(L):
      x = self.layer(x)
    return x

def tokenize(word, max_length=12):
  result = [ord(c) - ord('a') + 1 for c in word]
  result += [0] * (max_length - len(result))
  return torch.tensor(result).long()


In [315]:
all_data = [
  (tokenize(word), torch.tensor([val]))
  for i in range(1, 7)
  for word, val in data[i].items()
]
